In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import gradio as gr
from transformers import pipeline


2025-08-21 23:35:41.447273: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755819341.786687      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755819341.880713      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


******summarization******

In [3]:
# Use a pipeline as a high-level helper

pipe = pipeline("summarization", model="facebook/bart-large-cnn")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [4]:
def summarize_text(text, max_length=130, min_length=30):
    summary = pipe(text, max_length=max_length, min_length=min_length, do_sample=False)
    return summary[0]['summary_text']

In [5]:
text = """
Climate change is no longer a distant threat but a present reality with profound implications for global economies. Rising temperatures, extreme weather events, and shifting climate patterns are disrupting industries, supply chains, and communities worldwide. Agriculture is one of the sectors most affected, as unpredictable rainfall patterns and prolonged droughts threaten food security and increase production costs. Similarly, coastal cities face rising sea levels that put infrastructure, housing, and trade hubs at risk, leading to significant economic losses. Insurance companies are also grappling with the increasing costs of natural disasters, as the frequency and intensity of hurricanes, floods, and wildfires continue to rise. In response, many governments and corporations are investing heavily in renewable energy, green technologies, and sustainable practices to mitigate the damage and adapt to new environmental conditions. The transition to a low-carbon economy, while costly in the short term, is expected to create new opportunities for growth, innovation, and job creation. Industries such as solar and wind energy, electric vehicles, and carbon capture are rapidly expanding, driven by both environmental concerns and market demand. However, this transition also presents challenges, particularly for countries and industries heavily reliant on fossil fuels. The shift requires not only financial investment but also workforce retraining, policy reforms, and global cooperation. Economists warn that failure to address climate change could result in catastrophic financial losses, potentially surpassing those caused by global recessions or pandemics. On the other hand, proactive measures could transform climate risk into economic opportunity, fostering resilience and long-term prosperity.
"""
print(summarize_text(text))

Rising temperatures, extreme weather events, and shifting climate patterns are disrupting industries, supply chains, and communities worldwide. Many governments and corporations are investing heavily in renewable energy, green technologies, and sustainable practices. The transition to a low-carbon economy is expected to create new opportunities for growth, innovation, and job creation.


*****question-answering*****

In [6]:
# Use a pipeline as a high-level helper

pipel = pipeline("question-answering", model="distilbert/distilbert-base-cased-distilled-squad")

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [7]:
def answer_question(question, context):
    """
    Returns the answer to a question given a context.
    """
    result = pipel(question=question, context=context)
    return result['answer']

In [8]:
context_text = """
What do solar panels do?"""

print(answer_question("What is AI focused on?", context_text))

solar panels


In [9]:
def summarize_text(context):
    summary = pipe(context, max_length=130, min_length=30, do_sample=False)
    return summary[0]['summary_text']

def answer_question(context, question):
    result = pipel(question=question, context=context)
    return result['answer']

def combined_function(context, question, summarize_first):
    if summarize_first:
        context = summarize_text(context)
    answer = answer_question(context, question)
    return context, answer

# Gradio interface
interface = gr.Interface(
    fn=combined_function,
    inputs=[
        gr.Textbox(label="Context", lines=7, placeholder="Paste your text here..."),
        gr.Textbox(label="Question", lines=1, placeholder="Enter your question here..."),
        gr.Checkbox(label="Summarize context before answering?", value=False)
    ],
    outputs=[
        gr.Textbox(label="Processed Context"),
        gr.Textbox(label="Answer")
    ],
    title="Summarization + Question Answering App",
    description="Summarize your text and ask questions using BART and DistilBERT."
)

interface.launch()

* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://1a29158692c8b66859.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
